In [1]:
import pandas as pd
import json
import nltk
from nltk.probability import FreqDist
from funciones import get_sent_tokens,get_lemma,get_stems,remove_stop_words,get_word_tokens, remove_special_characters
data = pd.read_csv("../data/BooksDataSet.csv").drop(columns=['Unnamed: 0'])
data

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alvaro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,book_id,book_name,genre,summary
0,3248537,Drowned Wednesday,Fantasy,Drowned Wednesday is the first Trustee among ...
1,27796919,The Lost Hero,Fantasy,"As the book opens, Jason awakens on a school ..."
2,3910776,The Eyes of the Overworld,Fantasy,Cugel is easily persuaded by the merchant Fia...
3,5969644,Magic's Promise,Fantasy,The book opens with Herald-Mage Vanyel return...
4,3173445,Taran Wanderer,Fantasy,Taran and Gurgi have returned to Caer Dallben...
...,...,...,...,...
2995,10372180,White Death,Thriller,"A Novel from the NUMA files, A Kurt Austin Ad..."
2996,14504372,Venus with Pistol,Thriller,Gilbert Kemp is dealer specializing in antiqu...
2997,3617412,Blackwater,Thriller,"""How do you know when you're in too deep? Dav..."
2998,11320975,The Rainbow and the Rose,Thriller,The story concerns the life of Johnnie Pascoe...


In [2]:
data.isnull().sum()

book_id      0
book_name    0
genre        0
summary      0
dtype: int64

In [3]:
data.genre.value_counts()

genre
Fantasy             500
Science Fiction     500
Crime Fiction       500
Historical novel    500
Horror              500
Thriller            500
Name: count, dtype: int64

In [4]:
data2 = pd.read_csv("../data/booksummaries.txt", delimiter="\t", header=None)
data2.columns = ["id","random", "book_name", "author", "date_of_publish", "genres", "summary"]
data2

,id,random,book_name,author,date_of_publish,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
...,...,...,...,...,...,...,...
16554,36934824,/m/0m0p0hr,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,2000-06-01,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...
16556,37122323,/m/0n5236t,Decoded,Jay-Z,2010-11-16,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...
16557,37132319,/m/0n4bqb1,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [5]:
#necesitamos que esté el género y quitamos columnas que no son útiles
data2 = data2.dropna(subset="genres").drop(columns=["id", "random"])

In [6]:
def fix_genre(row):
    return json.loads(row.genres).values()

data2.loc[:,"genres"] = data2.apply(fix_genre, axis=1)
data2

,book_name,author,date_of_publish,genres,summary
0,Animal Farm,George Orwell,1945-08-17,"(Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,1962,"(Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,1947,"(Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...
4,A Fire Upon the Deep,Vernor Vinge,NaN,"(Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...
5,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"(War novel, Roman à clef)","The book tells the story of Paul Bäumer, a Ge..."
...,...,...,...,...,...
16549,The Third Lynx,Timothy Zahn,2007,(Science Fiction),The story starts with former government agent...
16551,Remote Control,Andy McNab,1997,"(Thriller, Fiction, Suspense)",The series follows the character of Nick Ston...
16555,Transfer of Power,Vince Flynn,2000-06-01,"(Thriller, Fiction)",The reader first meets Rapp while he is doing...
16556,Decoded,Jay-Z,2010-11-16,(Autobiography),The book follows very rough chronological ord...


In [7]:
data2 = data2.explode("genres").reset_index(drop=True)
data2

,book_name,author,date_of_publish,genres,summary
0,Animal Farm,George Orwell,1945-08-17,Roman à clef,"Old Major, the old boar on the Manor Farm, ca..."
1,Animal Farm,George Orwell,1945-08-17,Satire,"Old Major, the old boar on the Manor Farm, ca..."
2,Animal Farm,George Orwell,1945-08-17,Children's literature,"Old Major, the old boar on the Manor Farm, ca..."
3,Animal Farm,George Orwell,1945-08-17,Speculative fiction,"Old Major, the old boar on the Manor Farm, ca..."
4,Animal Farm,George Orwell,1945-08-17,Fiction,"Old Major, the old boar on the Manor Farm, ca..."
...,...,...,...,...,...
29999,Transfer of Power,Vince Flynn,2000-06-01,Thriller,The reader first meets Rapp while he is doing...
30000,Transfer of Power,Vince Flynn,2000-06-01,Fiction,The reader first meets Rapp while he is doing...
30001,Decoded,Jay-Z,2010-11-16,Autobiography,The book follows very rough chronological ord...
30002,Poor Folk,Fyodor Dostoyevsky,1846,Epistolary novel,Makar Devushkin and Varvara Dobroselova are s...


In [13]:
data2.groupby('genres').count()

,book_name,author,date_of_publish,summary
genres,,,,
Absurdist fiction,29,28,23,29
Adventure,23,18,13,23
Adventure novel,330,316,276,330
Albino bias,2,2,2,2
Alien invasion,1,1,1,1
...,...,...,...,...
Wuxia,18,17,16,18
Young adult literature,825,736,679,825
Youth,1,0,1,1


In [8]:
def summary_processing(row):
    summary = row["summary"]
    get_stems(summary)
    

In [12]:
a = data2.summary[0]
sentences = get_sent_tokens(a)
clean = remove_special_characters(sentences)
words = get_word_tokens(clean)
stems = get_stems(words)
lemma = get_lemma(stems)
pre = remove_stop_words(lemma)
pre

['old',
 'major',
 'old',
 'boar',
 'manor',
 'farm',
 'call',
 'anim',
 'farm',
 'meet',
 'compar',
 'human',
 'parasit',
 'teach',
 'anim',
 'revolutionari',
 'song',
 'beast',
 'england',
 'major',
 'die',
 'two',
 'young',
 'pig',
 'snowbal',
 'napoleon',
 'assum',
 'command',
 'turn',
 'hi',
 'dream',
 'philosophi',
 'anim',
 'revolt',
 'drive',
 'drunken',
 'irrespons',
 'mr',
 'jone',
 'farm',
 'renam',
 'anim',
 'farm',
 'adopt',
 'seven',
 'command',
 'anim',
 'import',
 'anim',
 'equal',
 'snowbal',
 'attempt',
 'teach',
 'anim',
 'read',
 'write',
 'food',
 'plenti',
 'farm',
 'run',
 'smoothli',
 'pig',
 'elev',
 'themselv',
 'posit',
 'leadership',
 'set',
 'asid',
 'special',
 'food',
 'item',
 'ostens',
 'person',
 'health',
 'napoleon',
 'take',
 'pup',
 'farm',
 'dog',
 'train',
 'privat',
 'napoleon',
 'snowbal',
 'struggl',
 'leadership',
 'snowbal',
 'announc',
 'hi',
 'plan',
 'build',
 'windmil',
 'napoleon',
 'ha',
 'hi',
 'dog',
 'chase',
 'snowbal',
 'away',
 '